In [5]:
import pandas as pd
import requests
from configparser import ConfigParser
import boto3
from botocore.exceptions import NoCredentialsError
import io
import json
import codecs

In [2]:
# Carregar as configurações do arquivo ini
config = ConfigParser()
config.read("../config/config.ini")

['../config/config.ini']

In [3]:
# Caminho para download dos dados
# url_municipios_ibge = config.get("URL", "municipios_ibge")
url_municipios_ibge = 'https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/json/municipios.json'
url_municipios_ibge

'https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/json/municipios.json'

In [6]:
# Fazer a requisição para obter o JSON da URL
response = requests.get(url_municipios_ibge)
# Decodificar o conteúdo da resposta removendo o BOM
content = response.content.decode('utf-8-sig')
municipios_data = json.loads(content)

In [7]:
print(municipios_data[0])  # Exemplo: imprimir o primeiro município

{'codigo_ibge': 5200050, 'nome': 'Abadia de Goiás', 'latitude': -16.7573, 'longitude': -49.4412, 'capital': 0, 'codigo_uf': 52, 'siafi_id': 1050, 'ddd': 62, 'fuso_horario': 'America/Sao_Paulo'}


In [8]:
len(municipios_data)

5570

In [9]:
# Carregar o JSON em um DataFrame Pandas
df = pd.json_normalize(municipios_data)

In [10]:
df.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [11]:
# Configurações para conexão com o MinIO/S3
endpoint = config.get("MinIO", "endpoint")
access_key = config.get("MinIO", "access_key")
secret_key = config.get("MinIO", "secret_key")
bucket_raw = config.get("Bucket", "bucket_raw")
bucket_context = config.get("Bucket", "bucket_context")
prefix_municipios_ibge_json = config.get("Bucket", "prefix_municipios_ibge_json")
prefix_municipios_ibge_csv = config.get("Bucket", "prefix_municipios_ibge_csv")

In [12]:
print("endpoint = ", endpoint)
print("access_key = ", access_key)
print("secret_key = ", secret_key)
print("bucket_raw = ", bucket_raw)
print("bucket_context = ", bucket_context)
print("prefix_municipios_ibge_json = ", prefix_municipios_ibge_json)
print("prefix_municipios_ibge_csv = ", prefix_municipios_ibge_csv)

endpoint =  http://minio:9000
access_key =  aulafia
secret_key =  aulafia@123
bucket_raw =  raw
bucket_context =  context
prefix_municipios_ibge_json =  municipios_ibge_json/
prefix_municipios_ibge_csv =  municipios_ibge_csv/


In [13]:
# Inicializar o cliente boto3 para S3
s3_client = boto3.client(
    "s3",
    endpoint_url=endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
)

In [14]:
# Armazenar o JSON no bucket 'raw'
json_filename = "MunicipiosIBGE.json"
json_s3_path = f"{prefix_municipios_ibge_json}{json_filename}"

# try:
#     s3_client.put_object(Bucket=s3_bucket_raw, Key=json_s3_path, Body=response.content)
#     print(f"JSON file uploaded to MinIO: {json_s3_path}")
# except NoCredentialsError:
#     print("Credentials not available.")

s3_client.put_object(Bucket=bucket_raw, Key=prefix_municipios_ibge_json + json_filename, Body=response.content)

{'ResponseMetadata': {'RequestId': '177F51EEC108F88A',
  'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '0',
   'etag': '"f4c04b20ccf2bfc9bc71d22a22a30d06"',
   'server': 'MinIO',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8',
   'x-amz-request-id': '177F51EEC108F88A',
   'x-content-type-options': 'nosniff',
   'x-xss-protection': '1; mode=block',
   'date': 'Sun, 27 Aug 2023 18:47:52 GMT'},
  'RetryAttempts': 0},
 'ETag': '"f4c04b20ccf2bfc9bc71d22a22a30d06"'}

In [15]:
s3_client.close()